# Question 72 - LC 106. Construct Binary Tree from Inorder and Postorder Traversal

Given two integer arrays `inorder` and `postorder` where `inorder` is the inorder traversal of a binary tree and `postorder` is the postorder traversal of the same tree, construct and return _the binary tree_.

**Example 1:**

    Input: inorder = [9,3,15,20,7], postorder = [9,15,7,20,3]
    Output: [3,9,20,null,null,15,7]

**Example 2:**

    Input: inorder = [-1], postorder = [-1]
    Output: [-1]

**Constraints:**

- `1 <= inorder.length <= 3000`
- `postorder.length == inorder.length`
- `-3000 <= inorder[i], postorder[i] <= 3000`
- `inorder` and `postorder` consist of unique values.
- Each value of `postorder` also appears in `inorder`.
- `inorder` is guaranteed to be the inorder traversal of the tree.
- `postorder` is guaranteed to be the postorder traversal of the tree.


In [1]:
from __future__ import annotations
from typing import Optional
from collections import deque

In [2]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    @staticmethod
    def from_list(values: list[Optional[int]]) -> Optional[TreeNode]:
        """
        Constructs a binary tree from a list of values using level-order traversal (BFS).

        Parameters:
        - values (list): The list of values to construct the binary tree from.

        Returns:
        TreeNode: The root of the constructed binary tree.
        """
        if not values:
            return None

        root = TreeNode(val=values[0])
        queue = deque([root])
        i = 1

        while queue and i < len(values):
            current_node = queue.popleft()

            if values[i] is not None:
                current_node.left = TreeNode(val=values[i])
                queue.append(current_node.left)
            i += 1

            if i < len(values) and values[i] is not None:
                current_node.right = TreeNode(val=values[i])
                queue.append(current_node.right)
            i += 1

        return root

    def print_level_order(self):
        """
        Prints the level order traversal of the tree (BFS).

        Returns:
        list: A list of lists, where each sublist contains the values of the tree nodes
              at that depth.
        """
        queue = deque([(self, 0)])
        result = []
        current_level = []
        level_number = 0

        while queue:
            current_node, node_level = queue.popleft()

            if node_level > level_number:
                result.append(current_level)
                current_level = []
                level_number = node_level

            if current_node:
                current_level.append(current_node.val)
                queue.append((current_node.left, node_level + 1))
                queue.append((current_node.right, node_level + 1))
            else:
                current_level.append("None")

        if current_level:
            result.append(current_level)
        
        # Remove trailing "None" values
        result = [level for level in result if any(elem != "None" for elem in level)]

        return result

In [3]:
class Solution:
    # divide and conquer - find root in postorder, then find left and right subtrees in inorder
    def buildTree(self, inorder: list[int], postorder: list[int]) -> Optional[TreeNode]:
        if not inorder or not postorder:
            return None

        # the last element in postorder is the root
        root_val = postorder.pop()
        root = TreeNode(root_val)

        # find the index of the root in inorder
        root_index = inorder.index(root_val)

        # find the right subtree in inorder
        root.right = self.buildTree(inorder[root_index + 1:], postorder)
        root.left = self.buildTree(inorder[:root_index], postorder)

        return root

In [4]:
# Test case

s = Solution()

root = s.buildTree([9, 3, 15, 20, 7], [9, 15, 7, 20, 3])
root.print_level_order()

[[3], [9, 20], ['None', 'None', 15, 7]]

The reason for constructing the right subtree first in the solution, especially in the context of using the `postorder` traversal, is rooted in the structure of the `postorder` array itself. Let's break down the reasoning:

### Postorder Traversal

In postorder traversal, a tree is traversed in the following order: left subtree, right subtree, and then the root node. This means that the last element in a postorder sequence is the current tree's root node. The elements before the root are divided into two groups: all the elements belonging to the left subtree come first, followed by the elements belonging to the right subtree.

### Why Build the Right Subtree First?

Given how we pop the last element of the `postorder` list to determine the root, it's crucial to manage the remaining elements in the `postorder` array correctly, as they represent the left and right subtrees. Since the elements for the right subtree appear right before the root element in the postorder sequence, and we are popping the root element from the end of the `postorder` list, the sequence of elements that remains in the `postorder` list naturally aligns with the expectation for building the right subtree first:

- After popping the root, the end of the `postorder` array directly aligns with the right subtree elements.
- By recursively calling the function to build the right subtree first, we effectively consume the right subtree elements from the `postorder` list, which is crucial because the size of the right subtree dictates how many elements we should take from the end of the `postorder` list.
- Once the right subtree is built and those elements are consumed, the `postorder` list now ends with the elements belonging to the left subtree, correctly positioning us to build the left subtree in the next recursive call.

### The Recursive Calls

The recursive construction is done in two parts, each time slicing the `inorder` array to pass only the relevant portion for the subtree being constructed:

- `root.right = buildTree(inorder[root_index + 1:], postorder)`: For the right subtree, we use the part of the `inorder` array to the right of the root's position. This is because, in the inorder sequence, all elements to the right of the root belong to the right subtree.
- `root.left = buildTree(inorder[:root_index], postorder)`: For the left subtree, we use the part of the `inorder` array to the left of the root's position, consistent with how inorder traversal works (elements to the left of the root belong to the left subtree).

This methodical approach, considering the nature of `postorder` and `inorder` traversals, ensures the binary tree is reconstructed accurately from the given traversal sequences.

Let's use the example given in the question to illustrate why we build the right subtree first when reconstructing a binary tree from inorder and postorder traversals.

### Example:

- **Inorder traversal**: `[9,3,15,20,7]`
- **Postorder traversal**: `[9,15,7,20,3]`

### Understanding the Traversal Orders:

- **Inorder**: Left subtree → Node → Right subtree
- **Postorder**: Left subtree → Right subtree → Node

### Steps to Reconstruct the Tree:

#### 1. Identify the Root:

- The last item in the postorder list is the root of the tree. In this case, `3` is the root.

#### 2. Locate the Root in the Inorder List:

- Find `3` in the inorder list. It divides the tree into the left and right subtrees.
- **Inorder left subtree**: `[9]`
- **Inorder right subtree**: `[15,20,7]`

#### 3. Build Right Subtree First:

- Why right first? Because when we pop `3` (the root) from the postorder, we're left with `[9,15,7,20]`, which ends with the right subtree's nodes in postorder.

#### Example Breakdown for Right Subtree:

- **Remaining Postorder**: `[9,15,7,20]` (after removing `3`)
- **Right Subtree Inorder**: `[15,20,7]`
- The last element in the remaining postorder (`20`) is the root of the right subtree.
- In the right subtree's inorder (`[15,20,7]`), `20` divides it into `[15]` (left) and `[7]` (right).

By building the right subtree first, we utilize the postorder's natural ordering. After determining the right subtree, the postorder array directly corresponds to what's left to process, fitting perfectly for constructing the left subtree.

#### Constructing the Right Subtree:

- **Root**: `20`
- **Left Child**: Found by locating `20` in the inorder array and taking everything to the left of it, which is `[15]`. In postorder, `15` is immediately before `20`, so it matches.
- **Right Child**: Everything to the right of `20` in the inorder array, which is `[7]`. Since `7` is before `20` in postorder (after removing elements related to the left child and the root), it aligns with the construction order.

#### Constructing the Left Subtree:

- Now, we're left with `[9]` in both inorder and postorder for the left subtree, clearly indicating that `9` is a leaf node on the left.

### Visual Representation:

After building both subtrees, we have the tree:

```
     3
    / \
   9  20
     /  \
    15   7
```

- The process starts from the root found at the end of postorder.
- Building the right subtree first leverages the postorder array's natural order after the root is removed.
- This approach, rooted in the structure of inorder and postorder traversals, ensures that we correctly reconstruct the tree by systematically consuming the postorder array to reflect the true structure of the tree.